In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Telus_resi_customers='''



with ADC_customer_base as(

SELECT customer_id,dealer_customer_id,dealer_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) ='2022-08-31'
and account_type_name!='Standalone'
and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id)

select * from ADC_customer_base as adc
inner join Telus_customers as telus
on telus.cust_bus_cust_id=adc.dealer_customer_id


'''

In [ ]:
Telus_Customer_DF=extract_bq_data(bq_client, sql=Telus_resi_customers)

In [ ]:
Telus_Customer_DF.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADC_Telus_Customer_31Aug2022'

bq_table_instance= bq_client.load_table_from_dataframe(Telus_Customer_DF, Table_BQ,job_config=config)

In [ ]:
ADT_Customer_Base='''


select * from `divgpras-pr-579355.ADC.ADC_ADT_mapping`


'''

In [ ]:
ADT_Customer_Base=extract_bq_data(bq_client, sql=ADT_Customer_Base)

In [ ]:
ADT_Customer_Base.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADT_Customer_base'

bq_table_instance= bq_client.load_table_from_dataframe(ADT_Customer_Base, Table_BQ,job_config=config)

In [ ]:
ADT_resi_customers='''



with ADC_customer_base as(

SELECT customer_id,dealer_customer_id,dealer_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) ='2022-08-31'
and account_type_name!='Standalone'
and customer_type_name='Customer'
and dealer_name='ADT by TELUS'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id
)
,

ADT_customers as
(select * from `divgpras-pr-579355.ADC_updated.ADT_Customer_base` )

select * from ADC_customer_base as adc
inner join ADT_customers as ADT
on ADT.customer_id=adc.customer_id


''' 

In [ ]:
ADT_Customer_DF=extract_bq_data(bq_client, sql=ADT_resi_customers)

In [ ]:
ADT_Customer_DF.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADC_ADT_Customer_31Aug2022'

bq_table_instance= bq_client.load_table_from_dataframe(ADT_Customer_DF, Table_BQ,job_config=config)

In [ ]:
All_resi_customers= '''


with ADC_Customer_Base as (

SELECT *
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) ='2022-08-31'
and date(join_date)<'2022-08-01'
and account_type_name!='Standalone'
and customer_type_name='Customer'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id )



, Telus_Customer_Base as (

SELECT customer_id,'Telus_Customer' as Dealer_type_flag,COUNT(*) OVER (PARTITION BY 1) as RowCnt 
FROM `divgpras-pr-579355.ADC_updated.ADC_Telus_Customer_31Aug2022` 
)


, ADT_Customer_Base as (

SELECT customer_id,'ADT_Customer' as Dealer_type_flag,COUNT(*) OVER (PARTITION BY 1) as RowCnt 
FROM `divgpras-pr-579355.ADC_updated.ADC_ADT_Customer_31Aug2022` 
)


, Union_data as (

SELECT * from Telus_Customer_Base
UNION ALL
SELECT * from ADT_Customer_Base

)

,Merge_data as (

SELECT * from ADC_Customer_Base a
INNER JOIN Union_data b

on a.customer_id=b.customer_id
order by Dealer_type_flag
)

select * from Merge_data


'''

In [ ]:
All_resi_DF=extract_bq_data(bq_client, sql=All_resi_customers)

In [ ]:
All_resi_DF.info()

In [ ]:
All_resi_DF.head()

In [ ]:
All_resi_DF['dealer_name'].value_counts()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADC_resi_customers_base_31Aug2022'

bq_table_instance= bq_client.load_table_from_dataframe(All_resi_DF, Table_BQ,job_config=config)

In [ ]:
ADC_Telus_internet_customers='''

with  Telus_internet_customers as

(select cust_bus_cust_id, pi_prod_instnc_typ_cd as pi_prod_instnc_typ_cd_internet
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_stat_cd in ('A')
and pi_prod_instnc_typ_cd ='HSIC'
and consldt_cust_typ_cd = 'R'
)

,
Telus_customers as
(select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id)
,

ADC_customer_base as(

SELECT customer_id,dealer_customer_id,dealer_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) ='2022-08-31'
and account_type_name!='Standalone'
and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id

)
,
merge_data as(

select distinct adc.customer_id,adc.dealer_customer_id, adc.dealer_name,telus.bacct_bus_bacct_num, telus.bacct_billg_acct_id, telus.cust_bus_cust_id,
       telus.pi_prod_instnc_typ_cd, telus.pi_prod_instnc_stat_ts, telus.prod_instnc_ts,b.cust_bus_cust_id as cust_bus_cust_id_internet ,
       b.pi_prod_instnc_typ_cd_internet 
from ADC_customer_base adc
inner join Telus_customers telus
on telus.cust_bus_cust_id=adc.dealer_customer_id
inner join Telus_internet_customers b
on adc.dealer_customer_id=b.cust_bus_cust_id
order by telus.bacct_bus_bacct_num
)




select distinct customer_id, dealer_customer_id, dealer_name,
       bacct_bus_bacct_num, bacct_billg_acct_id, cust_bus_cust_id,
       pi_prod_instnc_typ_cd, pi_prod_instnc_stat_ts, prod_instnc_ts,cust_bus_cust_id_internet,
       pi_prod_instnc_typ_cd_internet, 1 as Telus_internet_flag 
       from merge_data


'''

In [ ]:
ADC_Telus_internet=extract_bq_data(bq_client, sql=ADC_Telus_internet_customers)

In [ ]:
ADC_Telus_internet.info()

In [ ]:
ADC_Telus_internet.head()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.ADC_Telus_internet_customers_31Aug2022'

bq_table_instance= bq_client.load_table_from_dataframe(ADC_Telus_internet, Table_BQ,job_config=config)